# **Householder Notebook**

@eti-calde

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

## **1.1 Householder Reflection**

**Definition**
Given a unit vector $u$ defining a hyperplane perpendicular to ir, a Householder reflection over a vector $x$ is defined as:

$$
H_{\mathbf{u}}(x) = x-2 \langle \mathbf{x},\mathbf{u}\rangle \mathbf{u}
$$

Is nothing more that the vector reflected over the hyperplane. This can be expressed in matrix form as follows:

$$
H =(I-2\mathbf{u}^T\mathbf{u})
$$

---

## **1.2 let’s see the transformation in code!!**

In [ ]:
def get_householder_matrix(v):
    """
    Calculates the Householder reflection matrix for a vector v
    
    Parameters
    ----------
    v : ndarray (n,), input vector.
    Returns
    -------
    H : ndarray (n,n), Householder reflection matrix.
    """
    v = v.reshape(-1, 1)
    I = np.eye(v.shape[0])
    return I - 2 * (v @ v.T) / (v.T @ v)

In [ ]:
cmap = plt.get_cmap("magma")
C_V1   = cmap(0.15)  # x
C_V2   = cmap(0.35)  # u
C_PLN  = cmap(0.85)  # hyperplane
C_REF  = cmap(0.60)  # H(x)

def plot_householder(theta1_deg=30.0, theta2_deg=120.0, r1=1.5):
    # Parameters
    t1 = np.deg2rad(theta1_deg)
    t2 = np.deg2rad(theta2_deg)
    unitari_rad = 1.0

    # Vectors
    x = np.array([r1*np.cos(t1), r1*np.sin(t1)])
    u = np.array([unitari_rad*np.cos(t2), unitari_rad*np.sin(t2)])

    # Hyperplane
    n_perp = np.array([-u[1], u[0]])

    # Householder
    H = get_householder_matrix(u)
    x_householder = H @ x
    print("x:", x)
    print("x' (Householder):", x_householder)
    # ===================================================================
    # PLOT
    plt.figure()

    # Vectors
    plt.quiver(0, 0, x[0], x[1], color=C_V1, scale=1, scale_units='xy', angles='xy', label='x')
    plt.quiver(0, 0, u[0], u[1], color=C_V2, scale=1, scale_units='xy', angles='xy', label='u (unitario)')
    plt.quiver(0, 0, x_householder[0], x_householder[1],
               color=C_REF, scale=1, scale_units='xy', angles='xy', label='H(x)')
    # Hyperplane
    L = 2.2
    pts = np.vstack([-L*n_perp, L*n_perp])
    plt.plot(pts[:,0], pts[:,1], linestyle='--', color=C_PLN, label='Hip. u·x = 0')

    # Details
    lim = 2.2
    plt.xlim(-lim, lim); plt.ylim(-lim, lim)
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title("Householder reflection of x")
    plt.show()

angle1 = FloatSlider(value=30.0,  min=0.0, max=360.0, step=1.0, description='Ángulo x (°)')
angle2 = FloatSlider(value=120.0, min=0.0, max=360.0, step=1.0, description='Ángulo u (°)')
len_r1 = FloatSlider(value=1.5,   min=0.0, max=2.0,   step=0.1, description='||x||')

interact(plot_householder, theta1_deg=angle1, theta2_deg=angle2, r1=len_r1)


Now let's apply the Householder Reflection to get the QR factorization of a matrix $A$.

---

## **2.3 QR Factorization**

In [ ]:
def get_Q_Matrix(householder_vectors):
    """
    Calculates Q matrix of QR factorization using Householder reflections.

    Parameters
    ----------
    householder_vectors : ndarray (n,n), Householder reflection vectors.
    each i-th column represents a Householder vector for the i-th step of the QR factorization.

    Returns
    -------
    Q : ndarray (n,n), Orthogonal matrix.
    """
    n = householder_vectors.shape[0]
    Q = np.eye(n)
    for i in range(n):
        u = householder_vectors[:n-i, i]
        H_tilde = get_householder_matrix(u)
        H_full = np.eye(n)
        H_full[i:, i:] = H_tilde
        Q = Q @ H_full
    return Q

def QR_Householder(A):
    """
    QR factorization using Householder reflections.

    Parameters
    ----------
    A : ndarray (n,n), input matrix.
    Returns
    -------
    Q,R : ndarray (n,n), Orthogonal matrix and upper triangular matrix.
    """
    if (A.shape[1] != A.shape[0]):
        print("Matrix is not square")
        return None, None
    
    n = A.shape[0]
    # We need to work with submatrices of A
    # and we don't want to modify the original Matrix
    A = A.copy()

    # Each Householder vector is 1 dimension smaller than the previous one,
    # So we use just the first n-i rows on each iteration 
    householder_vectors = np.zeros((n, n))

    for i in range(n):
        sub_A = A[i:, i:]
        # Get Column i of A
        col_i_of_A = sub_A[:,0]
        col_i_of_A = col_i_of_A.reshape(-1,1)

        # Compute the Householder vector
        alpha = -np.sign(col_i_of_A[0,0]) * np.linalg.norm(col_i_of_A)
        e1 = np.eye(col_i_of_A.shape[0])[:,0].reshape(-1,1)
        u = col_i_of_A - alpha * e1
        u = u / np.linalg.norm(u)
        # print(f"u en la iteracion {i}, {u.shape} =\n", u)

        # Save the Householder vector
        householder_vectors[0:n-i,i] = u.ravel()
        # print(f"holder_vectors en la iteracion {i} =\n", holder_vectors)
        
        # FIX RESPECTO A LA VERSIÓN DE LA CALSE ANTERIOR.
        A[i:, i:] -= 2 * np.outer(u, u.T @ A[i:, i:])

    Q = get_Q_Matrix(householder_vectors)
    R = np.triu(A)
    # R = A
    return Q,R

In [ ]:
size = 4
A = np.random.rand(size,size) + np.identity(size)*2
print("Matriz A\n",A)
Q,R = QR_Householder(A)
# print("uni?:\n", Q.T@Q)
compare_numpy = True
if (size <= 5):
    # print("A =\n", A)
    print("R:\n", R)
    print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A - Q @ R))
    if compare_numpy:
        A_np, Q_np, R_np = A.copy(), Q.copy(), R.copy()
        print("Comparando con numpy...")
        Q_np, R_np = np.linalg.qr(A_np)
        print("R (numpy):\n", R_np)
        print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A_np - Q_np @ R_np))
else:
    print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A - Q @ R))
    if compare_numpy:
        A_np, Q_np, R_np = A.copy(), Q.copy(), R.copy()
        print("Comparando con numpy...")
        Q_np, R_np = np.linalg.qr(A_np)
        print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A_np - Q_np @ R_np))

---
## **2.4 Arnoldi Householder**

### Idea Principal

El resultado del algoritmo de Arnoldi se basa en la reducción parcial de $A$ a una forma Hesenberg:
$$
A V_k = V_{k+1} \tilde{H}_k \tag{1}
$$

Recordemos que $V_k$ es el espacio que deseamos ortonormalizar qu esta compuesto por los vectores del sub espacio de Krylov:
$$
V_k = \{r_0, A r_0, A^2 r_0, \dots, A^{k-1}r_0\}
$$

Ahora notamos que el lado izquierdo de la ecuación **(1)** $A V_k$ podemos escribir la multiplicación así:
$$
A V_k = \{Ar_0, A^2  r_0, A^3 r_0, \dots, A^{k}r_0\}
$$

Lo cual es muy cercano a $V_{k+1}$, pero nos faltaría agregar la primera columna al espacio que es $r_0$, llegando a:
$$
[r_0 A V_k] = V_{k+1} R_k \tag{2}
$$

Donde:
$$
R_k =
\begin{bmatrix}
\beta  & h_{1,1} & \dots  & h_{1,k} \\
0      & h_{2,1} & \dots  & h_{2,k} \\
\vdots & \vdots & \ddots & \vdots \\
0      & 0      & \dots  & h_{k+1,k}
\end{bmatrix}
$$

El objetivo de hacer esto es que ahora en la ecucación **(2)** tenemos una matriz del lado izquierdo $[r_0 A V_k]$ y del lado derecho una matriz ortogonal $V_{k+1}$ y una matriz triangular superior $R_k$ de decir podemos llegar al lado derecho aplicando QR al lado izquierdo.

### Detalles

Para aplicar Arnoldi en GMRes no se parte de la base completa $V_k$, mas bien se construye en cada iteración un nuevo vector de la base, por lo cual surge la duda ¿cómo se obtiene el vector $v_i$?

1. El plan es hacer factorización QR sobre $[r_0 A V_k]$:
$$
[r_0 A V_k] = Q R
$$

donde $Q = V_{k+1}$ porque la construcción viene de la reducción parcial de Hessenberg y por lo aprendido en QR Householder, sabemos que:
$$
V_{k+1} = Q = P_1 P_2 \dots P_{k+1}
$$

Las columnas de la matriz $V_{k+1}$ son los $v_i$ que andamos buscando y ahora es sencillo que que si queremos obenter la i-ésima columna podemos:

$$
v_i = V_{k+1} e_i = (P1 P2 \dots P_{k+1}) e_i
$$

Es más aún, dada la estructura de las matrices $P$, recordad que cada una es una identidad con una submatriz dentro, por lo tanto:

$$
v_i = P_1 P_2 \dots P_j e_j
$$





In [ ]:
size = 5
A = np.random.rand(size, size)
e3 = np.zeros(size)
e3[2] = 1
print(A)
print("\n",A @ e3)

In [ ]:
def get_householder_vector(x):
    if np.linalg.norm(x) < 1e-16:
        return np.zeros_like(x)
    
    alpha = np.linalg.norm(x)
    e1 = np.zeros_like(x)
    e1[0] = 1.0
    
    # El signo garantiza la estabilidad numérica evitando la resta de números cercanos
    v = x + np.copysign(alpha, x[0]) * e1
    v /= np.linalg.norm(v)
    return v

# =============== TEST ===============

# rand_vec = np.random.rand(3)
# norm = np.linalg.norm(rand_vec)
# householder_vec = get_householder_vector(rand_vec)
# apply_householder = rand_vec - 2 * householder_vec * (householder_vec.T @ rand_vec)
# e1 = np.zeros(3)
# e1[0] = 1
# print("original vec:\n", rand_vec)
# print("apply householder vec:\n", apply_householder)
# print("|rand vec| * e1:\n", norm * e1)

# =====================================


In [ ]:
def apply_householder_sequence(x, householder_vecs, reverse=False):
    vec = x.copy()
    iterator = reversed(householder_vecs) if reverse else householder_vecs
    for w in iterator:
        vec = vec - 2 * w * (w.T @ vec)     
    return vec

In [ ]:
def householder_arnoldi(A, r0, m, verbose=False):
    debug = lambda msg: print(msg)  if verbose else None

    n = A.shape[0]
    V = np.zeros((n, m + 1))
    H_bar = np.zeros((m + 1, m))
    householder_vecs = []
    z = r0.copy() # Vectors to transform to form H_bar

    rhs_scalar = 0.0
    # Main loop from j=1 to m+1
    for j in range(1, m + 2):
        debug(f"\n =============== ITERARTION j={j} =============== ")
        debug(f"\n =================== STEP 1 =================== ")     
        debug(f"1. Get householder vector w_{j} hat send z to |z|e1")    
        
        sub_z       = z[j - 1:]
        v           = get_householder_vector(sub_z)        
        w_j         = np.zeros(n)
        w_j[j - 1:] = v
        householder_vecs.append(w_j)
        
        debug(f"2. Householder Vector w_{j}:\n {w_j}")                    
        debug(f"\n ================================================ \n")  


        debug(f"\n =================== STEP 2 =================== ")      
        debug("1. Apply the householder transformation w_{j} to z ")      
        debug("We do not update H_bar on the first iteration") if j==1 else None

        h_full = z - 2 * w_j * (w_j.T @ z)
        if j > 1 and j <= m + 1:
            H_bar[:, j - 2] = h_full[:m + 1]
            debug(f"2. H_bar updated (column {j - 2}):\n {H_bar}\n")   
        elif j ==1:
            rhs_scalar = h_full[0]
        debug(f"\n ================================================ \n")  
        
        
        debug(f"\n =================== STEP 3 =================== ")      
        debug("1. Construct the new vector of V applying the householder sequence")   
        e_j = np.zeros(n)
        e_j[j - 1] = 1.0
        v_j = apply_householder_sequence(e_j, householder_vecs, reverse=True)
        V[:, j - 1] = v_j
        
        debug(f"2. new v_{j} = \n{v_j}")
        debug(f"3. add v_{j} a V[:, {j - 1}] \n{V}")
        debug(f"\n ================================================ \n")


        if j <= m:
            debug(f"\n =================== STEP 4 =================== ")   
            debug("1. Prepare z for the next iteration")
            Av = A @ v_j
            z = apply_householder_sequence(Av, householder_vecs, reverse=False)
            debug(f"new vector z_{j+1} = A v_{j} and then apply old reflections  = \n{z}\n")
            debug(f"\n ================================================ \n")

    V_m = V[:, :m]
    V_m_plus_1 = V

    if verbose:
        debug(f"\n =================== EXTRA CHECKS =================== ")
        debug("1. Check the orthonormality of V_m: ")
        debug(f"| I - V_m^T @ V_m | :\n {np.linalg.norm(np.identity(m)- (V_m.T @ V_m))}")
        LHS = A @ V_m
        RHS = V_m_plus_1 @ H_bar
        error = np.linalg.norm(LHS - RHS)
        debug("2. Verify Arnoldi's fundamental identity ")
        debug(f"\nError ||A*V_m - V_{{m+1}}*H_bar||: {error:.2e}")

    return V_m, V_m_plus_1, H_bar, rhs_scalar


np.random.seed(0)
n = 3
A = np.random.rand(n, n)
r0 = np.random.rand(n)
r0 = r0 / np.linalg.norm(r0)
m = 2

V_m, V_m_plus_1, H_bar, _ = householder_arnoldi(A, r0, m, verbose=True)



# GMRes Householder LSTQ

In [ ]:
def gmres_householder_lstsq(A, b, m, tol=1e-10, verbose=False):
    n = A.shape[0]
    
    # --- INICIALIZACIÓN DE GMRES ---
    x0 = np.zeros(n)
    r0 = b - A @ x0
    beta = np.linalg.norm(r0)
    r0 = r0 / beta
    
    residuals_history = [beta]
    
    if beta < tol:
        return x0, residuals_history

    V_m, V_m_plus_1, H_bar, rhs_escalar = householder_arnoldi(A, r0, m, verbose=False)
    print(rhs_escalar)
        
    # 1. Definir el lado derecho del sistema de mínimos cuadrados
    # DETALLE DE VITAL IMPORTANCIA Y DONDE PERDÍ HORASSSS :c
    # Householder no manda v1 (v1 = r0/beta) a (||v1|| e1)
    # Puede ser a (||v1|| * +e1) o (||v1|| * -e1)
    rhs = np.zeros(m + 1)
    rhs[0] = rhs_escalar * beta
    
    # 2. Resolver el sistema para encontrar y
    y, _, _, _ = np.linalg.lstsq(H_bar, rhs, rcond=None)
    
    # 3. Calcular la solución final actualizando x0
    x_final = x0 + V_m @ y
    
    # Cálculo del historial de residuos para análisis
    for k in range(1, m):
        yk, _, _, _ = np.linalg.lstsq(H_bar[:k+1, :k], rhs[:k+1], rcond=None)
        xk = x0 + V_m_plus_1[:, :k] @ yk
        residuals_history.append(np.linalg.norm(b - A @ xk))
    
    residuals_history.append(np.linalg.norm(b - A @ x_final))

    return x_final, residuals_history

np.random.seed(42)
n = 10
A = np.random.rand(n, n) + np.identity(n) * 2
x = np.ones(n)
b = A @ x
m = 8

x_sol, res_hist = gmres_householder_lstsq(A, b, m)

final_residual_norm = np.linalg.norm(b - A @ x_sol)
print(f"Solución encontrada: {x_sol}")
print("\nHistorial de la norma del residuo (calculado a posteriori):")
for i, res in enumerate(res_hist):
    print(f"  Iteración {i}: {res:.4e}")

---

# Gmres Householde Givens\#2


In [30]:
def givens_rotation(a,b):
    """
    Calcula los valores de seno y coseno para una rotación de Givens.
    """
    if b == 0:
        c = 1.0
        s = 0.0
    else:
        hyp = np.hypot(a, b)
        c = a / hyp
        s = -b / hyp
    return c, s

In [ ]:
from scipy.linalg import hessenberg

def givens_rotation(a,b):
    """
    Calcula los valores de seno y coseno para una rotación de Givens.
    """
    if b == 0:
        c = 1.0
        s = 0.0
    else:
        hyp = np.hypot(a, b)
        c = a / hyp
        s = -b / hyp
    return c, s

def create_random_hessenberg_matrix(n: int) -> np.ndarray:
    if n < 3:
        raise ValueError("La dimensión debe ser un entero positivo.")
    
    A = np.random.rand(n, n)
    H = hessenberg(A)
    return H

def get_rotaciones_hessenberg(H):
    n = H.shape[0]
    R = H.copy().astype(np.float64)

    c_values =[]
    s_values = []
    for row in range(n - 1):
        diagonal = R[row, row]
        subdiagonal = R[row + 1, row]

        c, s = givens_rotation(diagonal, subdiagonal)

        c_values.append(c)
        s_values.append(s)
        for column in range(row, n):
            temp_diag = R[row, column]
            temp_subdiag = R[row + 1, column]
            R[row, column] = c * temp_diag - s * temp_subdiag
            R[row + 1, column] = s * temp_diag + c * temp_subdiag


    return c_values, s_values

def apply_givens_sequence(c_values, s_values, H):
    """
    Aplica una secuencia de rotaciones de Givens (definida por c y s)
    a una matriz H.
    """
    # Creamos una copia para no modificar la matriz original
    R = H.copy().astype(np.float64)
    n = R.shape[0]

    # Iteramos a través de cada par (c, s) para aplicar su rotación
    for i in range(len(c_values)):
        c = c_values[i]
        s = s_values[i]

        # La rotación i-ésima se aplica a las filas i e i+1.
        # La transformación se aplica a todos los elementos de esas dos filas.
        for j in range(n):
            # Guardamos los valores actuales de las dos filas en la columna j
            temp_row1 = R[i, j]
            temp_row2 = R[i + 1, j]

            # Aplicamos la rotación y actualizamos los valores en la matriz
            R[i, j]       = c * temp_row1 - s * temp_row2
            R[i + 1, j]   = s * temp_row1 + c * temp_row2

    return R

# H_example = create_random_hessenberg_matrix(4)

# print("Matriz de Hessenberg original:")
# print(np.round(H_example,2))
# print("\n" + "-"*30 + "\n")

# c,s = get_rotaciones_hessenberg(H_example)

# c2 = c[:2]
# s2 = s[:2]

# H_2 = apply_givens_sequence(c2,s2,H_example)

# print("Matriz triangular superior resultante:")
# print(np.round(H_2,2))

Matriz de Hessenberg original:
[[ 0.8  -0.43 -0.88 -0.42]
 [-0.84  1.09  0.41  0.04]
 [ 0.    0.7   0.79  0.16]
 [ 0.    0.    0.7   0.05]]

------------------------------

Matriz triangular superior resultante:
[[ 1.16 -1.08 -0.9  -0.32]
 [ 0.    0.83  0.48 -0.01]
 [ 0.    0.    0.72  0.32]
 [ 0.    0.    0.7   0.05]]


In [ ]:
def gmres_householder_givens(A, b, x0, m, tol=1e-10, verbose=False):
    debug = lambda msg: print(msg)  if verbose else None

    # ====================== GMRES INITIALIZATION ======================
    n = A.shape[0]
    r0 = b - A @ x0
    beta = np.linalg.norm(r0)    
    r0 = r0 / beta
    residuals = [beta]
    if beta < tol:
        return x0, residuals

    # ===================== ARNOLDI INITIALIZATION =====================
    V = np.zeros((n, m + 1))
    H_bar = np.zeros((m + 1, m))
    householder_vecs = []
    z = r0.copy()   # Vecs to transform to form H_bar
    rhs_scalar = 0.0
    
    # ===================== GIVENS INITIALIZATION =====================
    c = np.zeros(m)
    s = np.zeros(m)
    # g = np.zeros(m + 1) 
    
    # Main loop from j=1 to m+1
    for j in range(1, m + 2):
        debug(f"\n =============== ITERARTION j={j} =============== ")
        debug(f"\n =================== STEP 1 =================== ")     
        debug(f"1. Get householder vector w_{j} hat send z to |z|e1")    
        
        sub_z       = z[j - 1:]
        v           = get_householder_vector(sub_z)        
        w_j         = np.zeros(n)
        w_j[j - 1:] = v
        householder_vecs.append(w_j)
        
        debug(f"2. Householder Vector w_{j}:\n {w_j}")                    
        debug(f"\n ================================================ \n")  


        debug(f"\n =================== STEP 2 =================== ")      
        debug("1. Apply the householder transformation w_{j} to z ")      
        debug("We do not update H_bar on the first iteration") if j==1 else None

        h_full = z - 2 * w_j * (w_j.T @ z)
        if j > 1 and j <= m + 1:
            H_bar[:, j - 2] = h_full[:m + 1]
            debug(f"2. H_bar updated (column {j - 2}):\n {H_bar}\n")   
        elif j ==1:
            rhs_scalar = h_full[0]
        debug(f"\n ================================================ \n")  
        
        
        debug(f"\n =================== STEP 3 =================== ")      
        debug("1. Construct the new vector of V applying the householder sequence")   
        e_j = np.zeros(n)
        e_j[j - 1] = 1.0
        v_j = apply_householder_sequence(e_j, householder_vecs, reverse=True)
        V[:, j - 1] = v_j
        
        debug(f"2. new v_{j} = \n{v_j}")
        debug(f"3. add v_{j} a V[:, {j - 1}] \n{V}")
        debug(f"\n ================================================ \n")


        if j <= m:
            debug(f"\n =================== STEP 4 =================== ")   
            debug("1. Prepare z for the next iteration")
            Av = A @ v_j
            z = apply_householder_sequence(Av, householder_vecs, reverse=False)
            debug(f"new vector z_{j+1} = A v_{j} and then apply old reflections  = \n{z}\n")
            debug(f"\n ================================================ \n")

        if j>=1:
            debug(f"\n =================== STEP 5 =================== ")
            debug("1. Get Givens Rotation to make H_bar triangular")
            c[j-1], s[j-1] = givens_rotation(H_bar[j-1,j-1],H_bar[j,j-1])
            G_H = apply_givens_sequence(c[:j-1], s[j-1], H_bar, j-1)
            debug(f"This is the original H_bar:\n {H_bar}")
            

    # V_m = V[:, :m]
    # V_m_plus_1 = V

    rhs = np.zeros(m + 1)
    rhs[0] = rhs_scalar * beta
    
    # 2. Resolver el sistema para encontrar y
    y, _, _, _ = np.linalg.lstsq(H_bar, rhs, rcond=None)
    
    # 3. Calcular la solución final actualizando x0
    x_final = x0 + V_m @ y
    
    # Cálculo del historial de residuos para análisis
    for k in range(1, m):
        yk, _, _, _ = np.linalg.lstsq(H_bar[:k+1, :k], rhs[:k+1], rcond=None)
        xk = x0 + V_m_plus_1[:, :k] @ yk
        residuals_history.append(np.linalg.norm(b - A @ xk))
    
    residuals_history.append(np.linalg.norm(b - A @ x_final))

    return x_final, residuals_history



# --- EJEMPLO DE USO ---
np.random.seed(42)
n = 10
A = np.random.rand(n, n) + np.identity(n) * 2
b = np.random.rand(n)
x0 = np.zeros(n)
m = 8 # Máximo de iteraciones

# Ejecutar el solver
x_sol, residuals_history = gmres_householder_givens(A, b, x0, m, tol=1e-8, verbose=True)

# Verificar la solución
final_residual_norm = np.linalg.norm(b - A @ x_sol)
print(f"\nSolución encontrada. Norma del residuo final: {final_residual_norm:.4e}")
print("Historial de la norma del residuo:")
for i, res in enumerate(residuals_history):
    print(f"  Iteración {i}: {res:.4e}")